## Импорт и установка библиотек

In [1]:
# !pip install dbfread # file reader
# !pip install patool # rar unarchiver

In [1]:
import pandas as pd
import patoolib
import os
import numpy as np
import shutil

from dbfread import DBF

In [2]:
def read_dbf(file):
    return pd.DataFrame(iter(DBF(file, encoding='cp866')))

## Разархивирование файлов

## Чтение файлов

### Формат файлов отчета 0409802
- Файл PK802yymm.DBF, где yymm – отчетный год, отчетный месяц: файлы «Консолидированный балансовый отчет»
- Файл PS802yymm.DBF, где yymm – отчетный год, отчетный месяц: файлы «Консолидированный балансовый отчет» раздела «Справочно»

In [6]:
# костыль, чтоб нормально внеслись данные за разные года

df_balance = pd.DataFrame()

for filename in sorted(os.listdir('data/rars/0409802')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/0409802/{filename}', outdir=f'data/0409802/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/0409802/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('PK')][0]
        frame = read_dbf(f'data/0409802/{name}/{filename}')
        
        date = name.split('-')[1]
        
        frame.drop(columns=['KORR_P', 'KORR_M', 'VSEGO'], inplace=True) 

        df_balance = pd.concat([df_balance, frame], join='outer').drop_duplicates()
        shutil.rmtree(f'data/0409802/{name}')

shutil.rmtree(f'data/0409802')
    
df_balance = df_balance.reset_index(drop=True).sort_values(['REGN_GKO', 'STR'])

INFO patool: ... creating output directory `data/0409802/802-20180101'.
INFO patool: ... creating output directory `data/0409802/802-20180401'.
INFO patool: ... creating output directory `data/0409802/802-20180701'.
INFO patool: ... creating output directory `data/0409802/802-20181001'.
INFO patool: ... creating output directory `data/0409802/802-20190101'.
INFO patool: ... creating output directory `data/0409802/802-20190401'.
INFO patool: ... creating output directory `data/0409802/802-20190701'.
INFO patool: ... creating output directory `data/0409802/802-20191001'.
INFO patool: ... creating output directory `data/0409802/802-20200101'.
INFO patool: ... creating output directory `data/0409802/802-20200401'.
INFO patool: ... creating output directory `data/0409802/802-20200701'.
INFO patool: ... creating output directory `data/0409802/802-20201001'.
INFO patool: ... creating output directory `data/0409802/802-20210101'.
INFO patool: ... creating output directory `data/0409802/802-202

In [7]:
for filename in sorted(os.listdir('data/rars/0409802')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/0409802/{filename}', outdir=f'data/0409802/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/0409802/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('PK')][0]
        frame = read_dbf(f'data/0409802/{name}/{filename}')
        
        date = name.split('-')[1]
        frame = frame.rename(columns={
            # если оставлять корректировки, то закомментировать нижнюю строку и раскомментировать верхние
            #'KORR_P': 'KORR_P_'+date,
            #'KORR_M': 'KORR_M_'+date,
            #'VSEGO': 'VSEGO'+date
            'VSEGO': date
        })

        # оставляем только колонку "ВСЕГО", если оставляем корректировки, то закомментировать строку
        frame.drop(columns=['KORR_P', 'KORR_M'], inplace=True) 

        df_balance = df_balance.merge(frame, how='left')
        shutil.rmtree(f'data/0409802/{name}')

shutil.rmtree(f'data/0409802')
    
df_balance = df_balance.reset_index(drop=True)

INFO patool: ... creating output directory `data/0409802/802-20171001'.
INFO patool: ... creating output directory `data/0409802/802-20180101'.
INFO patool: ... creating output directory `data/0409802/802-20180401'.
INFO patool: ... creating output directory `data/0409802/802-20180701'.
INFO patool: ... creating output directory `data/0409802/802-20181001'.
INFO patool: ... creating output directory `data/0409802/802-20190101'.
INFO patool: ... creating output directory `data/0409802/802-20190401'.
INFO patool: ... creating output directory `data/0409802/802-20190701'.
INFO patool: ... creating output directory `data/0409802/802-20191001'.
INFO patool: ... creating output directory `data/0409802/802-20200101'.
INFO patool: ... creating output directory `data/0409802/802-20200401'.
INFO patool: ... creating output directory `data/0409802/802-20200701'.
INFO patool: ... creating output directory `data/0409802/802-20201001'.
INFO patool: ... creating output directory `data/0409802/802-202

In [8]:
df_balance

,REGN_GKO,STR,20171001,20180101,20180401,20180701,20181001,20190101,20190401,20190701,...,20200401,20200701,20201001,20210101,20210401,20210701,20211001,20230701,20231001,20240101
0,1,1,10635086.0,11771672.0,9896604.0,10036567.0,9089270.0,12091355.0,10358528.0,10851933.0,...,19294002.0,17295651.0,16142639.0,14704211.0,13068725.0,12620484.0,10708255.0,21037937.0,28341362.0,21420053.0
1,1,10,23171855.0,26010212.0,23841457.0,24095925.0,23580097.0,25511387.0,7378077.0,7404857.0,...,8346413.0,8059469.0,7826953.0,7487232.0,7494913.0,7260470.0,7082979.0,7096683.0,6468995.0,7003594.0
2,1,10.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,10.2,NaN,NaN,NaN,NaN,NaN,NaN,7378077.0,7404857.0,...,8346413.0,8059469.0,7826953.0,7487232.0,7494913.0,7260470.0,7082979.0,7096683.0,6468995.0,7003594.0
4,1,10.3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7716,3532,7,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
7717,3532,8,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7718,3532,8.1,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7719,3532,9,NaN,NaN,NaN,NaN,0.0,0.0,41830.0,49611.0,...,67621.0,72820.0,76342.0,74168.0,74002.0,70192.0,66063.0,61088.0,NaN,NaN


In [9]:
df_balance.to_csv('data/data_balances.csv')

### Формат файлов отчета 0409803
- Файл PK803yymm.DBF, где yymm – отчетный год, отчетный месяц: файл «Консолидированный отчет о финансовых результатах»

In [10]:
# костыль, чтоб нормально внеслись данные за разные года

df_pnl = pd.DataFrame()

for filename in sorted(os.listdir('data/rars/0409803')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/0409803/{filename}', outdir=f'data/0409803/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/0409803/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('PK')][0]
        frame = read_dbf(f'data/0409803/{name}/{filename}')
        
        date = name.split('-')[1]

        frame.drop(columns=['KORR_P', 'KORR_M', 'VSEGO'], inplace=True) 

        df_pnl = pd.concat([df_pnl, frame], join='outer').drop_duplicates()
        shutil.rmtree(f'data/0409803/{name}')

shutil.rmtree(f'data/0409803')
    
df_pnl = df_pnl.reset_index(drop=True).sort_values(['REGN_GKO', 'STR'])

INFO patool: ... creating output directory `data/0409803/803-20171001'.
INFO patool: ... creating output directory `data/0409803/803-20180101'.
INFO patool: ... creating output directory `data/0409803/803-20180401'.
INFO patool: ... creating output directory `data/0409803/803-20180701'.
INFO patool: ... creating output directory `data/0409803/803-20181001'.
INFO patool: ... creating output directory `data/0409803/803-20190101'.
INFO patool: ... creating output directory `data/0409803/803-20190401'.
INFO patool: ... creating output directory `data/0409803/803-20190701'.
INFO patool: ... creating output directory `data/0409803/803-20191001'.
INFO patool: ... creating output directory `data/0409803/803-20200101'.
INFO patool: ... creating output directory `data/0409803/803-20200401'.
INFO patool: ... creating output directory `data/0409803/803-20200701'.
INFO patool: ... creating output directory `data/0409803/803-20201001'.
INFO patool: ... creating output directory `data/0409803/803-202

In [11]:
for filename in sorted(os.listdir('data/rars/0409803')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/0409803/{filename}', outdir=f'data/0409803/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/0409803/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('PK')][0]
        frame = read_dbf(f'data/0409803/{name}/{filename}')
        
        date = name.split('-')[1]
        frame = frame.rename(columns={
            # если оставлять корректировки, то закомментировать нижнюю строку и раскомментировать верхние
            #'KORR_P': 'KORR_P_'+date,
            #'KORR_M': 'KORR_M_'+date,
            #'VSEGO': 'VSEGO'+date
            'VSEGO': date
        })

        # оставляем только колонку "ВСЕГО", если оставляем корректировки, то закомментировать строку
        frame.drop(columns=['KORR_P', 'KORR_M'], inplace=True) 

        df_pnl = df_pnl.merge(frame, how='left')
        shutil.rmtree(f'data/0409803/{name}')

shutil.rmtree(f'data/0409803')
    
df_pnl = df_pnl.reset_index(drop=True)

INFO patool: ... creating output directory `data/0409803/803-20171001'.
INFO patool: ... creating output directory `data/0409803/803-20180101'.
INFO patool: ... creating output directory `data/0409803/803-20180401'.
INFO patool: ... creating output directory `data/0409803/803-20180701'.
INFO patool: ... creating output directory `data/0409803/803-20181001'.
INFO patool: ... creating output directory `data/0409803/803-20190101'.
INFO patool: ... creating output directory `data/0409803/803-20190401'.
INFO patool: ... creating output directory `data/0409803/803-20190701'.
INFO patool: ... creating output directory `data/0409803/803-20191001'.
INFO patool: ... creating output directory `data/0409803/803-20200101'.
INFO patool: ... creating output directory `data/0409803/803-20200401'.
INFO patool: ... creating output directory `data/0409803/803-20200701'.
INFO patool: ... creating output directory `data/0409803/803-20201001'.
INFO patool: ... creating output directory `data/0409803/803-202

In [12]:
df_pnl.to_csv('data/data_pnls.csv')

### Формат файлов "Консолидированная финансовая отчетность"
!!! по тому какая форма какой файл я не уверена 

- Файл F815yymm.DBF, где yymm – отчетный год, отчетный месяц: ОФП **=> дополнение в `df_balance`**
- Файл F816yymm.DBF, где yymm – отчетный год, отчетный месяц: ОПУ **=> дополнение в `df_pnl`**
- Файл F817yymm.DBF, где yymm – отчетный год, отчетный месяц: ОДДС

In [11]:
# костыль, чтоб нормально внеслись данные за разные года

df_pnl_early = pd.DataFrame()

for filename in sorted(os.listdir('data/rars/Консолидированная отчетность')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        #patoolib.extract_archive(f'data/rars/Консолидированная отчетность/{filename}', outdir=f'data/Консолидированная отчетность/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/rars/Консолидированная отчетность/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('F816')][0]
        frame = read_dbf(f'data/rars/Консолидированная отчетность/{name}/{filename}')
        
        date = name.split('-')[1]

        frame.drop(columns=['CP', 'NUM_PRIM', 'S_LAST'], inplace=True)

        df_pnl_early = pd.concat([df_pnl_early, frame], join='outer').drop_duplicates()
    
#df_pnl_early = df_pnl_early.reset_index(drop=True).sort_values(['REGN_GKO', 'STR'])

In [12]:
df_pnl_early

,REGN,RAZDEL,N_PP,N_PP_A,KOD,KOD_PRIZ,KOD_NAME,S_OTCH
0,2015,,1,1,81601.1,1,Процентные доходы,237664.0
1,2015,,2,2,81601.2,1,Процентные расходы,-101918.0
2,2015,,3,3,81601.3,1,Резерв под обесценение кредитов,-47321.0
3,2015,,4,4,81601.4,1,Чистые процентные доходы/(отрицательная процен...,88425.0
4,2015,,5,5,81601.5,1,Чистые непроцентные доходы,37426.0
...,...,...,...,...,...,...,...,...
4194,2602,,52,52,81602.10,1,Административные и прочие операционные расходы...,795566.0
4195,2602,,53,53,81602.10.1,1,расходы на персонал,399185.0
4196,2602,,54,54,81602.10.2,1,амортизация основных средств,51889.0
4197,2602,,55,55,81602.10.6,1,амортизация программного обеспечения и прочих ...,15149.0


In [13]:
df_pnl

,REGN_GKO,RAZDEL,STR,20171001,20180101,20180401,20180701,20181001,20190101,20190401,...,20200401,20200701,20201001,20210101,20210401,20210701,20211001,20230701,20231001,20240101
0,1,I,1,56768543.0,76718548.0,18881898.0,39248826.0,60994787.0,82313463.0,21422516.0,...,18700971.0,36334211.0,52353883.0,67353041.0,14234280.0,31325755.0,47414074.0,39984047.0,64244023.0,91547721.0
1,1,II,1,23646741.0,29553063.0,8212388.0,11026823.0,15169562.0,21033702.0,2177587.0,...,4528400.0,1936134.0,7144605.0,10436606.0,4663485.0,11021006.0,17733694.0,10737955.0,34097384.0,43600182.0
2,1,I,1.1,7500247.0,10239543.0,2691344.0,5971118.0,8999064.0,11540358.0,2905160.0,...,2043568.0,3531636.0,5348598.0,7193390.0,1746195.0,3578736.0,5454025.0,1055043.0,1872789.0,2896259.0
3,1,I,1.2,44671749.0,59867918.0,14309311.0,29653890.0,46342002.0,63028844.0,16497775.0,...,14559720.0,29016717.0,41876286.0,55258386.0,11752502.0,26269132.0,39928080.0,38690348.0,62086299.0,88323047.0
4,1,I,1.2.1,NaN,NaN,NaN,NaN,NaN,NaN,16497775.0,...,14559720.0,29016717.0,41876286.0,53386452.0,10860983.0,24423992.0,37020315.0,16462147.0,28971081.0,45823100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6236,3532,I,8,NaN,NaN,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6237,3532,II,8,NaN,NaN,NaN,NaN,0.0,0.0,1639.0,...,-1654.0,-938.0,-9318.0,2271.0,-634.0,NaN,NaN,NaN,NaN,NaN
6238,3532,I,8.1,NaN,NaN,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6239,3532,I,9,NaN,NaN,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Обработка показателей

Рассматриваемые показатели:

1) Отношение собственных средств (34) к чистым активам (34-21)
2) Отношение резервов (2.1.1 - обязательные, 20 - возможные потери, 31.1 - оценочные под ожидаемые кредитные убытки) к чистым активам (34-21)
3) Чистые активы (34-21) или их логарифм
4) _Отношение просроченной задолженности по ссудам к кредитам экономики_
5) _Отношение просроченной задолженности по ссудам к обязательным резервам в ЦБ РФ_
6) _Отношение оборотов по корреспондентским счетам за период времени к чистым активам_
7) Отношение балансовой прибыли (32) к чистым активам (34-21)
8) Отношение чистой прибыли (32.1) к чистым активам (34-21)
9) _Норматив текущей ликвидности_
10) _Отношение ликвидных активов к чистым активам_
11) _Отношение депозитов физических лиц до 30 дней к депозитам физических лиц_
12) _Отношение объема негосударственных ценных бумаг к чистым активам_

Справка по строкам: https://www.consultant.ru/document/cons_doc_LAW_444612/c1c3277496f8d7c26caf7152797c75ab73d1a385/

In [73]:
def choose_str(df, str, str_col='STR', regn_col='REGN_GKO'):
    return df[df.STR == str].drop(columns=str_col).set_index(regn_col)

In [78]:
net_income = choose_str(df_balance, '32.1') # чистая прибыль [прибыль (убыток) за отчетный период]
balance_income = choose_str(df_balance, '32') # балансовая прибыль [Прибыль (убыток)]

capital = choose_str(df_balance, '34') # собственные средства
reserves = choose_str(df_balance, '20') + choose_str(df_balance, '31.1') + choose_str(df_balance, '2.1.1')

liabilities = choose_str(df_balance, '21') # обязательства

net_assets = capital - liabilities # Активы-нетто (чистые активы) - стоимость капитала по рыночной цене минус долговые обязательства.

In [80]:
X1 = capital / net_assets
X2 = reserves / net_assets
X3 = np.log(net_assets) # логарифм
# X4
# X5
# X6
X7 = balance_income / net_assets
X8 = net_income / net_assets
# X9
# X10
# X11
# X12

/opt/anaconda3/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/opt/anaconda3/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


In [ ]:
# regressors = pd.DataFrame(df_balance['REGN_GKO'].unique().copy(), columns=['REGN_GKO'])